In [3]:
from dotenv import load_dotenv
from pprint import pprint
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()

True

In [4]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0
)

In [6]:
prompt_find = ChatPromptTemplate.from_messages(
    [
        (
            "system", "you are a helpful assistant.",
        ),
        (
            "human","Find the Bible verse {verse} in English Standard Version. Just give me the Verse."
        )
    ]
)

In [7]:
task_find = prompt_find | model | StrOutputParser()

In [10]:
verse = task_find.invoke(
    {
        "verse": "John 3:16"
    }
)

In [11]:
print(verse)

John 3:16 (ESV): "For God so loved the world, that he gave his only Son, that whoever believes in him should not perish but have eternal life."


In [14]:
results = {
    "results": {
        "Korean" : "???",
        "Spanish": "???",
    }
}

In [15]:
pprint(results)

{'results': {'Korean': '???', 'Spanish': '???'}}


In [17]:
prompt_translate = ChatPromptTemplate.from_messages(
    [
        (
            "system", "You are an Expert treanlator who translats English to {output_language}",
        ),
        (
            "human","Translate this text: {text}"
        )
    ]
)

In [18]:
task_korean = prompt_translate | model | StrOutputParser()

In [20]:
results["results"]["Korean"] = task_korean.invoke(
    {
        "output_language":"korean",
        "text" : verse
    }
)

In [21]:
pprint(results)

{'results': {'Korean': '요한복음 3:16 (개역개정판): "하나님이 세상을 이처럼 사랑하사 독생자를 주셨으니, 이는 그를 '
                       '믿는 자마다 멸망하지 않고 영생을 얻게 하려 하심이라."',
             'Spanish': '???'}}


In [23]:
task_spanish = prompt_translate | model | StrOutputParser()

In [24]:
results["results"]["Spanish"] = task_spanish.invoke(
    {
        "output_language" : "Spanish",
        "text" : verse
    }
)

In [25]:
pprint(results)

{'results': {'Korean': '요한복음 3:16 (개역개정판): "하나님이 세상을 이처럼 사랑하사 독생자를 주셨으니, 이는 그를 '
                       '믿는 자마다 멸망하지 않고 영생을 얻게 하려 하심이라."',
             'Spanish': 'Juan 3:16 (RVR): "Porque de tal manera amó Dios al '
                        'mundo, que ha dado a su Hijo unigénito, para que todo '
                        'aquel que en él cree, no se pierda, mas tenga vida '
                        'eterna."'}}


In [26]:
def pretty_out(results):
    out = f"""
[English to Korean]
{results["results"]["Korean"]}
[English to Spanish]
{results["results"]["Spanish"]}
"""

    return out

In [27]:
out = pretty_out(results)

In [28]:
print(out)


[English to Korean]
요한복음 3:16 (개역개정판): "하나님이 세상을 이처럼 사랑하사 독생자를 주셨으니, 이는 그를 믿는 자마다 멸망하지 않고 영생을 얻게 하려 하심이라."
[English to Spanish]
Juan 3:16 (RVR): "Porque de tal manera amó Dios al mundo, que ha dado a su Hijo unigénito, para que todo aquel que en él cree, no se pierda, mas tenga vida eterna."



Parallel

In [29]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableParallel

In [50]:
task_korean = RunnableLambda(
    lambda text : prompt_translate.format_prompt(text=text, output_language="korean")
)


In [51]:
task_spanish = RunnableLambda(
    lambda text : prompt_translate.format_prompt(text=text, output_language = "Spanish")
)


In [52]:
task_translate = RunnableParallel(
    results={
        "Korean" : task_korean | model | StrOutputParser(),
        "Spanish" : task_spanish | model | StrOutputParser()
    }
)

In [53]:
task_combine = RunnableLambda(
    lambda results : pretty_out(results)
)

In [54]:
chain_parallel = (
    prompt_find
    | model
    | StrOutputParser()
    | task_translate
    | task_combine
    
)

In [55]:
out = chain_parallel.invoke(
    {
        "verse" : "John 3:16"
    }
)

In [56]:
print(out)


[English to Korean]
요한복음 3:16 (개역개정판): "하나님이 세상을 이처럼 사랑하사 독생자를 주셨으니, 이는 그를 믿는 자마다 멸망하지 않고 영생을 얻게 하려 하심이라."
[English to Spanish]
Juan 3:16 (RVR): "Porque de tal manera amó Dios al mundo, que ha dado a su Hijo unigénito, para que todo aquel que en él cree, no se pierda, mas tenga vida eterna."



batch

In [57]:
from langchain_core.prompts import ChatPromptTemplate

In [58]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", "당신은 뛰어난 번역가입니다.",
        ),
        (
            "human", "다음 한글 문장을 영어로 번역해주세요. 번역된 글 외에는 넣지 말아주세요: {text}"
        )
    ]
)

In [59]:
from langchain.chains import LLMChain

In [60]:
chain = prompt | model

In [61]:
sentences = [
    {"text": "영화를 보고 싶다."},
    {"text": "커피가 땡긴다."},
    {"text": "여행을 갔다."},
    {"text": "하나님이 세상을 이처럼 사랑하사 독생자를 주셨으니 이는 그를 믿는 자마다 멸망하지 않고 영생을 얻게 하심이라"},
]

run batch

In [62]:
out = chain.batch(sentences)

In [63]:
for msg in out:
    msg.pretty_print()

================================== Ai Message ==================================

I want to watch a movie.
================================== Ai Message ==================================

I crave coffee.
================================== Ai Message ==================================

I went on a trip.
================================== Ai Message ==================================

For God so loved the world that He gave His only begotten Son, that whoever believes in Him should not perish but have everlasting life.
